# Transfer learning Skeleton

## Data Preprocessing 

In [ ]:
import matplotlib.pyplot as plt
import PIL.Image as Image

import numpy as np
import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds

### Data Acquisition

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'rock_paper_scissors',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
print("ds_info:\n", ds_info)
num_classes = ds_info.features['label'].num_classes
print("num_classes: ", num_classes)




### Data Normalization

In [ ]:
def normalize_img(image, label):

    # expected_image_shape 
    expected_image_shape = (224,224)


    # Normalizes images: float32
    # rescales pixel values between [0,1]
    # resize image to expected_image_shape

    ## image = tf.image.convert_image_dtype(image, dtype=tf.float16)/ 255.
    image = tf.cast(image, tf.float32) / 255.
    image = tf.image.resize(image, expected_image_shape)
    label = tf.cast(tf.one_hot(label, num_classes), tf.uint8)
    return image, label

### Data Visualization

In [ ]:
sample_data = ds_train.take(1)  # Only take a single example
print(sample_data)

for n in sample_data:
    sign = Image.fromarray(n[0].numpy())
    label = n[1].numpy()
    print('labels: rock:0, paper:1, scissors:2')
    print("label: " + str(label))
    display(sign)




#### Normilize data and revert it back but rescale the final image

In [ ]:
sample_data = sample_data.map(normalize_img)
print(sample_data)

for n in sample_data:
    image = np.uint8(n[0].numpy()*255) 


    sign = Image.fromarray(image)
    label = n[1].numpy()
    print('labels: rock:0, paper:1, scissors:2')
    print("label: " + str(label))
    display(sign)

### Data train test split


performed in one method when downloaded from tfds.load

### Set Batch & Epoch variables

In [ ]:
batch_size = 16 
epochs = 6

### Formating of Data

In [ ]:
# Data is cast to float32 and rescaled to between 0-1
# resized to expected model input_shape

ds_train = ds_train.map(normalize_img,num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
# divide in batches

ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)

ds_train_batch = ds_train.batch(batch_size, drop_remainder=True)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

print(ds_train)

### how to get 1 batch from the data

`one_batch = ds_train.take(1)

as_np = list(ds_train.take(1).as_numpy_iterator())`

### Inspect the Data

In [ ]:
sample_batch = ds_train_batch.take(1)  # Only take a single example
print(sample_batch)
print('\n')

image = list(sample_batch.as_numpy_iterator())[0][0][0]
label = list(sample_batch.as_numpy_iterator())[0][1][0]

image = np.uint8(image*255)
image = Image.fromarray(image)
print('labels: rock:0, paper:1, scissors:2')
print("label: " + str(label))
display(sign)

In [ ]:
#  when not batchwise seperated
sample_data = ds_train.take(1)  # Only take a single example
print(sample_data)

for n in sample_data:
    image = np.uint8(n[0].numpy()*255) 

    sign = Image.fromarray(image)
    label = n[1].numpy()
    print('labels: rock:0, paper:1, scissors:2')
    print("label: " + str(label))
    display(sign)

## Create/Download Base of model

### Decide on base model, (ex. EfficientNet-B0) - downloadable from tensorflow.hub

##### Decide what you want to "re-train", final layer and/or layers within the model

(Find expected image size) 

In [ ]:
classifier_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

expected_input_shape = (224,224) 

feature_extractor = hub.KerasLayer(
    classifier_url,
    input_shape=expected_input_shape+(3,), 
    trainable=False) # Can be True, "Fine tune"



### Test Base-model on a batch

## Feature Extraction
##### (Incase of deciding on a "re-train"-configuration for only the top-layer classifier)

#### Test the feature extractor

In [ ]:
one_batch = ds_train_batch.take(1)
as_np = list(one_batch.as_numpy_iterator())[0][0]

print(feature_extractor(as_np))


### Freeze the convolutional base

In [ ]:
feature_extractor.trainable = False

### Add a Classification Head

In [ ]:
efn_model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(num_classes, activation='softmax')
])


efn_model.summary()

### Test the untrained model

In [ ]:
as_np = ds_train.take(1)
for n in as_np:

    image = np.uint8(n[0].numpy()*255) 

    sign = Image.fromarray(image)
    label = n[1].numpy()
    print('labels: rock:0, paper:1, scissors:2')
    print("label: " + str(label))
    display(sign)




    sample = n[0][np.newaxis, ...]

    predictions = efn_model(sample)
    print(predictions)

    predicted_class = np.argmax(predictions[0], axis=-1)
    
    print('Classes: \n')
    print('rock: 1, paper: 2, scissors: 3')
    print(predicted_class)

### Compile model


In [ ]:
efn_model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['acc']
    )

#### Custom callbacks

In [ ]:
class CollectBatchStats(tf.keras.callbacks.Callback):
  def __init__(self):
    self.batch_losses = []
    self.batch_acc = []

  def on_batch_end(self, batch, logs=None):
    self.batch_losses.append(logs['loss'])
    self.batch_acc.append(logs['acc'])
    self.model.reset_metrics()

### Train the model

In [ ]:
batch_stats_callback = CollectBatchStats()

print(ds_train_batch)

history = efn_model.fit(ds_train_batch, 
                        epochs=2,
                        callbacks = [batch_stats_callback],
                        )

### Visualize Learning curves

In [ ]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel("Training Steps")
plt.ylim([0,2])
plt.plot(batch_stats_callback.batch_losses)

In [ ]:
plt.figure()
plt.ylabel("Accuracy")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(batch_stats_callback.batch_acc)

## Fine tuning
In the feature extraction experiment, you were only training a few layers on top of an MobileNet V2 base model. The weights of the pre-trained network were **not** updated during training.

One way to increase performance even further is to train (or "fine-tune") the weights of the top layers of the pre-trained model alongside the training of the classifier you added. The training process will force the weights to be tuned from generic feature maps to features associated specifically with the dataset.

Note: This should only be attempted after you have trained the top-level classifier with the pre-trained model set to non-trainable. If you add a randomly initialized classifier on top of a pre-trained model and attempt to train all layers jointly, the magnitude of the gradient updates will be too large (due to the random weights from the classifier) and your pre-trained model will forget what it has learned.

Also, you should try to fine-tune a small number of top layers rather than the whole MobileNet model. In most convolutional networks, the higher up a layer is, the more specialized it is. The first few layers learn very simple and generic features that generalize to almost all types of images. As you go higher up, the features are increasingly more specific to the dataset on which the model was trained. The goal of fine-tuning is to adapt these specialized features to work with the new dataset, rather than overwrite the generic learning.

### Un-freeze the top layers of the model


### Compile the model

### Continue training the model

### Visualize Learning curves

# Export Model

In [30]:
import time
t = time.time()

export_path = "C:/Users/sadhos/Documents/vansbro/tmp/saved_models/{}".format(int(t))
efn_model.save(export_path, save_format='tf')

export_path

INFO:tensorflow:Assets written to: C:/Users/sadhos/Documents/vansbro/tmp/saved_models/1590333259\assets
INFO:tensorflow:Assets written to: C:/Users/sadhos/Documents/vansbro/tmp/saved_models/1590333259\assets


'C:/Users/sadhos/Documents/vansbro/tmp/saved_models/1590333259'

## Confirm model, by reloading

In [31]:
reloaded = tf.keras.models.load_model(export_path)

In [ ]:
result_batch = model.predict(image_batch)
reloaded_result_batch = reloaded.predict(image_batch)

In [ ]:
abs(reloaded_result_batch - result_batch).max()